In [1]:
import time
import numpy as np
import logging
import data_utils
from TrajectoryDataset import TrajectoryDataset

from models import *

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.optim import AdamW

import train_utils

In [20]:
model = torch.load('TransformerModel5.pt')

sample_attention = model.model.encoder.layers[0].self_attn

print(sample_attention)

sample_attention_attn = sample_attention.attn
sample_attention_linears = sample_attention.linears

print(sample_attention_attn[-1].shape)

print(sample_attention_linears[-1].weight)


MultiHeadAttention(
  (linears): ModuleList(
    (0-3): 4 x Linear(in_features=512, out_features=512, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)
torch.Size([8, 50, 50])
Parameter containing:
tensor([[-0.0989, -0.0679, -0.0254,  ..., -0.0107, -0.0355,  0.0198],
        [-0.0529,  0.0107,  0.0245,  ..., -0.0204,  0.0333, -0.0366],
        [-0.0829,  0.1316,  0.1065,  ...,  0.0379, -0.0688, -0.1450],
        ...,
        [-0.1003, -0.0297,  0.0946,  ...,  0.0542, -0.0440, -0.0282],
        [-0.0594,  0.0536,  0.0495,  ..., -0.0109, -0.1294,  0.0612],
        [-0.0427, -0.0107,  0.0896,  ..., -0.0110,  0.0014, -0.0448]],
       device='cuda:0', requires_grad=True)
